# MODEL - B

##  using separate models for user embedding and exercise recommendation.


In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
from collections import defaultdict

In [2]:
# # Load and preprocess data
# users_df = pd.read_csv('users.csv')
# utterances_df = pd.read_csv('utterances.csv')
# errors_df = pd.read_csv('errors.csv')
# exercises_df = pd.read_csv('exercises.csv')
# user_progress_df = pd.read_csv('user_progress.csv')


In [3]:
# # Preprocess categorical data
# le_country = LabelEncoder()
# le_age_band = LabelEncoder()
# le_proficiency = LabelEncoder()
# le_cefr = LabelEncoder()

# users_df['country_encoded'] = le_country.fit_transform(users_df['country'])
# users_df['age_band_encoded'] = le_age_band.fit_transform(users_df['age_band'])
# users_df['proficiency_level_encoded'] = le_proficiency.fit_transform(users_df['proficiency_level'])
# users_df['cefr_level_encoded'] = le_cefr.fit_transform(users_df['cefr_level'])

# # Normalize numerical data
# scaler = StandardScaler()
# numerical_columns = ['overall_progress', 'total_practice_time', 'exercises_completed', 'daily_active_days', 
#                      'weekly_active_days', 'total_sessions', 'avg_session_duration', 'adaptive_difficulty', 
#                      'engagement_score']
# users_df[numerical_columns] = scaler.fit_transform(users_df[numerical_columns])





In [4]:
# # Create a dictionary to store user embeddings
# user_embeddings = {}

# # Define the User Embedding Model--CAN BE MODIFIED
# class UserEmbeddingModel(nn.Module):
#     def __init__(self, num_countries, num_age_bands, num_proficiency_levels, num_cefr_levels, embedding_dim=32):
#         super(UserEmbeddingModel, self).__init__()
#         self.country_embedding = nn.Embedding(num_countries, embedding_dim)
#         self.age_band_embedding = nn.Embedding(num_age_bands, embedding_dim)
#         self.proficiency_embedding = nn.Embedding(num_proficiency_levels, embedding_dim)
#         self.cefr_embedding = nn.Embedding(num_cefr_levels, embedding_dim)
#         self.fc = nn.Linear(embedding_dim * 4 + len(numerical_columns), embedding_dim)

#     def forward(self, country, age_band, proficiency, cefr, numerical_features):
#         country_emb = self.country_embedding(country)
#         age_band_emb = self.age_band_embedding(age_band)
#         proficiency_emb = self.proficiency_embedding(proficiency)
#         cefr_emb = self.cefr_embedding(cefr)
#         combined = torch.cat((country_emb, age_band_emb, proficiency_emb, cefr_emb, numerical_features), dim=1)
#         return self.fc(combined)

In [5]:

# num_countries = len(le_country.classes_)
# num_age_bands = len(le_age_band.classes_)
# num_proficiency_levels = len(le_proficiency.classes_)
# num_cefr_levels = len(le_cefr.classes_)

# user_embedding_model = UserEmbeddingModel(num_countries, num_age_bands, num_proficiency_levels, num_cefr_levels)
# optimizer = optim.AdamW(user_embedding_model.parameters())
# criterion = nn.MSELoss()

# # Train the embedding Model (simplified for brevity)
# for epoch in range(10):
#     for _, user in users_df.iterrows():
#         country = torch.tensor([user['country_encoded']])
#         age_band = torch.tensor([user['age_band_encoded']])
#         proficiency = torch.tensor([user['proficiency_level_encoded']])
#         cefr = torch.tensor([user['cefr_level_encoded']])
#         numerical = torch.tensor(user[numerical_columns].astype(np.float32).values).float().unsqueeze(0)
        
#         optimizer.zero_grad()
#         embedding = user_embedding_model(country, age_band, proficiency, cefr, numerical)
#         loss = criterion(embedding, torch.randn_like(embedding))  # Simplified loss
#         loss.backward()
#         optimizer.step()

#     print(f"Epoch {epoch+1}, Loss: {loss.item()}")


#     # BAD LOSS VALUES BECAUSE OF RANDOM TARGETS


Epoch 1, Loss: 1.7267507314682007
Epoch 2, Loss: 1.5114926099777222
Epoch 3, Loss: 1.0113203525543213
Epoch 4, Loss: 0.9759833812713623
Epoch 5, Loss: 0.6415071487426758
Epoch 6, Loss: 0.9812778234481812
Epoch 7, Loss: 1.2374804019927979
Epoch 8, Loss: 0.9554104804992676
Epoch 9, Loss: 0.860277533531189
Epoch 10, Loss: 1.232203722000122


In [6]:
# # Generate user embeddings
# for _, user in users_df.iterrows():
#     country = torch.tensor([user['country_encoded']])
#     age_band = torch.tensor([user['age_band_encoded']])
#     proficiency = torch.tensor([user['proficiency_level_encoded']])
#     cefr = torch.tensor([user['cefr_level_encoded']])
#     numerical = torch.tensor(user[numerical_columns].astype(np.float32).values).float().unsqueeze(0)
    
#     with torch.no_grad():
#         embedding = user_embedding_model(country, age_band, proficiency, cefr, numerical)
#     user_embeddings[user['user_id']] = embedding.squeeze().numpy()

In [7]:
#  #Define the Exercise Recommendation Model
# class ExerciseRecommendationModel(nn.Module):
#     def __init__(self, user_embedding_dim, hidden_dim, num_categories):
#         super(ExerciseRecommendationModel, self).__init__()
#         self.lstm = nn.LSTM(user_embedding_dim + 4, hidden_dim, batch_first=True)
#         self.attention = nn.MultiheadAttention(hidden_dim, num_heads=4, batch_first=True)
#         self.fc = nn.Linear(hidden_dim, num_categories)

#     def forward(self, user_embedding, error_history):
#         lstm_out, _ = self.lstm(torch.cat((user_embedding.unsqueeze(1).repeat(1, error_history.size(1), 1), error_history), dim=2))
#         attn_out, _ = self.attention(lstm_out, lstm_out, lstm_out)
#         return self.fc(attn_out[:, -1, :])

# # Prepare data for the Exercise Recommendation Model
# def prepare_error_history(user_id, timestamp, window_size=10):
#     user_errors = errors_df[(errors_df['user_id'] == user_id) & (errors_df['timestamp'] < timestamp)].sort_values('timestamp', ascending=False).head(window_size)
#     error_history = np.zeros((window_size, 4))
#     for i, (_, error) in enumerate(user_errors.iterrows()):
#         category_index = ['grammar', 'vocabulary', 'pronunciation', 'fluency'].index(error['category'])
#         error_history[i, category_index] = error['severity']
#     return error_history



In [8]:
# # Initialize and train the Exercise Recommendation Model
# num_categories = 4  # grammar, vocabulary, pronunciation, fluency
# recommendation_model = ExerciseRecommendationModel(32, 64, num_categories)
# optimizer = optim.Adam(recommendation_model.parameters())
# criterion = nn.CrossEntropyLoss()

# # Simplified training loop
# for epoch in range(10):
#     for _, exercise in exercises_df.iterrows():
#         user_id = exercise['user_id']
#         timestamp = exercise['timestamp']
#         user_embedding = torch.tensor(user_embeddings[user_id]).float().unsqueeze(0)
#         error_history = torch.tensor(prepare_error_history(user_id, timestamp)).float().unsqueeze(0)
        
#         optimizer.zero_grad()
#         prediction = recommendation_model(user_embedding, error_history)
#         target = torch.tensor([['grammar', 'vocabulary', 'pronunciation', 'fluency'].index(exercise['category'])])
#         loss = criterion(prediction, target)
#         loss.backward()
#         torch.nn.utils.clip_grad_norm_(recommendation_model.parameters(), 1.0)
#         optimizer.step()

#     print(f"Epoch {epoch+1}, Loss: {loss.item()}")


#     # BAD LOSS VALUES 

Epoch 1, Loss: 1.3223129510879517
Epoch 2, Loss: 1.3347889184951782
Epoch 3, Loss: 1.3286904096603394
Epoch 4, Loss: 1.3279350996017456
Epoch 5, Loss: 1.3246467113494873
Epoch 6, Loss: 1.3242452144622803
Epoch 7, Loss: 1.3075422048568726
Epoch 8, Loss: 1.2514656782150269
Epoch 9, Loss: 1.2273412942886353
Epoch 10, Loss: 1.156772255897522


In [9]:

# def recommend_exercise_category(user_id, timestamp):
#     user_embedding = torch.tensor(user_embeddings[user_id]).float().unsqueeze(0)
#     error_history = torch.tensor(prepare_error_history(user_id, timestamp)).float().unsqueeze(0)
    
#     with torch.no_grad():
#         prediction = recommendation_model(user_embedding, error_history)
    
#     category_index = prediction.argmax().item()
#     return ['grammar', 'vocabulary', 'pronunciation', 'fluency'][category_index]

# # Content and Format Recommendation--CAN BE MODIFIED
# def recommend_content_and_format(user_id, category):
#     user = users_df[users_df['user_id'] == user_id].iloc[0]
#     user_interests = user['interests'].split(', ')
#     user_country = user['country']
    
#     # Simple logic for content theme based on user interests and country
#     content_theme = np.random.choice(user_interests)
    
#     # Simple logic for exercise format - LABELS CAN BE MODIFIED
#     formats = ["fill-in-the-blank", "multiple-choice", "audio-recording", "sentence-formation"]
#     format_weights = [0.3, 0.3, 0.2, 0.2]
#     if category == 'pronunciation':
#         format_weights = [0.1, 0.1, 0.7, 0.1]
#     elif category == 'grammar':
#         format_weights = [0.4, 0.4, 0.1, 0.1]
    
#     exercise_format = np.random.choice(formats, p=format_weights)
    
#     return content_theme, exercise_format

# # Main recommendation function
# def recommend_exercise(user_id, timestamp):
#     category = recommend_exercise_category(user_id, timestamp)
#     content_theme, exercise_format = recommend_content_and_format(user_id, category)
    
#     return {
#         'category': category,
#         'content_theme': content_theme,
#         'format': exercise_format
#     }

In [17]:
# # Example usage

# errors_df['timestamp'] = pd.to_datetime(errors_df['timestamp'])

# user_id = users_df['user_id'].iloc[0]
# timestamp = pd.Timestamp.now()
# recommendation = recommend_exercise(user_id, timestamp)
# print(f"Recommended exercise for user {user_id}:")
# print(recommendation)

# # Function to update user embedding after completing an exercise
# def update_user_embedding(user_id, exercise_performance):
#     # this is a simplified update
#     current_embedding = user_embeddings[user_id]
#     performance_factor = exercise_performance['performance_score']
#     updated_embedding = current_embedding * (1 + 0.1 * performance_factor)
#     user_embeddings[user_id] = updated_embedding

# #simualtion
# def simulate_user_session(user_id, num_utterances=10):
#     print(f"Simulating session for user {user_id}")
#     for i in range(num_utterances):
#         timestamp = pd.Timestamp.now() + pd.Timedelta(minutes=i*10)
        
#         # Simulate user utterance (you would replace this with actual user input in a real system)
#         print(f"\nUtterance {i+1}")
        
#         # Decide whether to show an exercise (simplified logic as this algorithm was in place)
#         if np.random.random() < 0.6:  # 60% chance of showing an exercise
#             recommendation = recommend_exercise(user_id, timestamp)
#             print("Recommending exercise:")
#             print(recommendation)
            
           
#             performance_score = np.random.uniform(0.5, 1.0)
#             print(f"Simulated performance score: {performance_score:.2f}")
            
           
#             update_user_embedding(user_id, {'performance_score': performance_score})
#         else:
#             print("No exercise recommended for this utterance.")

# # Run simulation 
# random_user_id = np.random.choice(users_df['user_id'])
# simulate_user_session(random_user_id)

Recommended exercise for user e0fa472b-9738-4814-b366-5e0f6f668547:
{'category': 'fluency', 'content_theme': "'music'", 'format': 'audio-recording'}
Simulating session for user 6e43f566-f99a-40dd-a246-4c63ac6ee7af

Utterance 1
Recommending exercise:
{'category': 'fluency', 'content_theme': "['literature'", 'format': 'multiple-choice'}
Simulated performance score: 0.99

Utterance 2
No exercise recommended for this utterance.

Utterance 3
Recommending exercise:
{'category': 'fluency', 'content_theme': "['literature'", 'format': 'multiple-choice'}
Simulated performance score: 0.78

Utterance 4
Recommending exercise:
{'category': 'fluency', 'content_theme': "'anime']", 'format': 'fill-in-the-blank'}
Simulated performance score: 0.95

Utterance 5
Recommending exercise:
{'category': 'fluency', 'content_theme': "['literature'", 'format': 'fill-in-the-blank'}
Simulated performance score: 0.59

Utterance 6
Recommending exercise:
{'category': 'fluency', 'content_theme': "['literature'", 'format'

#  MODEL with attention mechanisms for processing utterance history.



In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

In [19]:

# users_df = pd.read_csv('users.csv')
# utterances_df = pd.read_csv('utterances.csv')
# errors_df = pd.read_csv('errors.csv')
# exercises_df = pd.read_csv('exercises.csv')
# user_progress_df = pd.read_csv('user_progress.csv')

In [20]:

# def preprocess_user_data(df):
#     le = LabelEncoder()
#     cat_columns = ['country', 'age_band', 'proficiency_level', 'cefr_level', 'current_focus']
#     for col in cat_columns:
#         df[col] = le.fit_transform(df[col])
    
#     # Handle list columns
#     df['interests'] = df['interests'].apply(eval)
#     df['personalized_goals'] = df['personalized_goals'].apply(eval)
    
#     scaler = StandardScaler()
#     num_columns = ['overall_progress', 'total_practice_time', 'exercises_completed', 'daily_active_days',
#                    'weekly_active_days', 'total_sessions', 'avg_session_duration', 'adaptive_difficulty',
#                    'engagement_score']
#     df[num_columns] = scaler.fit_transform(df[num_columns])
#     return df


# def preprocess_utterance_data(df):
#     scaler = StandardScaler()
#     num_columns = ['grammar_overall_score', 'grammar_error_count', 'vocabulary_overall_score', 'unique_words_count',
#                    'advanced_words_count', 'lexical_density', 'pronunciation_overall_score', 'phoneme_error_count',
#                    'intonation_score', 'rhythm_score', 'accent_similarity', 'fluency_overall_score', 'words_per_minute',
#                    'filler_word_count', 'pause_count', 'avg_pause_duration', 'longest_fluent_segment', 'coherence_score',
#                    'topic_relevance_score']
#     df[num_columns] = scaler.fit_transform(df[num_columns])
#     return df


# def preprocess_exercise_data(df):
#     le = LabelEncoder()
#     cat_columns = ['category', 'subcategory', 'format', 'content_theme']
#     for col in cat_columns:
#         df[col] = le.fit_transform(df[col])
    
#     scaler = StandardScaler()
#     num_columns = ['difficulty', 'performance_score', 'time_taken']
#     df[num_columns] = scaler.fit_transform(df[num_columns])
#     return df


# users_df = preprocess_user_data(users_df)
# utterances_df = preprocess_utterance_data(utterances_df)
# exercises_df = preprocess_exercise_data(exercises_df)


# def create_user_features(user):
#     return torch.tensor([
#         user['country'], user['age_band'], user['proficiency_level'], user['cefr_level'],
#         user['overall_progress'], user['total_practice_time'], user['exercises_completed'],
#         user['daily_active_days'], user['weekly_active_days'], user['total_sessions'],
#         user['avg_session_duration'], user['adaptive_difficulty'], user['engagement_score']
#     ], dtype=torch.float32)

# def create_utterance_features(utterance):
#     return torch.tensor([
#         utterance['grammar_overall_score'], utterance['grammar_error_count'],
#         utterance['vocabulary_overall_score'], utterance['unique_words_count'],
#         utterance['advanced_words_count'], utterance['lexical_density'],
#         utterance['pronunciation_overall_score'], utterance['phoneme_error_count'],
#         utterance['intonation_score'], utterance['rhythm_score'], utterance['accent_similarity'],
#         utterance['fluency_overall_score'], utterance['words_per_minute'],
#         utterance['filler_word_count'], utterance['pause_count'], utterance['avg_pause_duration'],
#         utterance['longest_fluent_segment'], utterance['coherence_score'],
#         utterance['topic_relevance_score']
#     ], dtype=torch.float32)

# def create_exercise_features(exercise):
#     return torch.tensor([
#         exercise['category'], exercise['subcategory'], exercise['format'],
#         exercise['difficulty'], exercise['performance_score'], exercise['time_taken']
#     ], dtype=torch.float32)

# # Create a dataset for training
# class ExerciseRecommendationDataset(Dataset):
#     def __init__(self, users, utterances, exercises, max_utterances=10):
#         self.users = users
#         self.utterances = utterances
#         self.exercises = exercises
#         self.max_utterances = max_utterances
        
#     def __len__(self):
#         return len(self.exercises)
    
#     def __getitem__(self, idx):
#         exercise = self.exercises.iloc[idx]
#         user_id = exercise['user_id']
#         user = self.users[self.users['user_id'] == user_id].iloc[0]
#         user_features = create_user_features(user)
        
#         user_utterances = self.utterances[self.utterances['user_id'] == user_id].tail(self.max_utterances)
#         utterance_features = torch.stack([create_utterance_features(u) for _, u in user_utterances.iterrows()])
        
#         # Pad utterance sequence if necessary
#         if len(utterance_features) < self.max_utterances:
#             padding = torch.zeros((self.max_utterances - len(utterance_features), utterance_features.size(1)))
#             utterance_features = torch.cat([utterance_features, padding], dim=0)
        
#         exercise_features = create_exercise_features(exercise)
        
#         return user_features, utterance_features, exercise_features

In [21]:
# # Model architecture
# class ExerciseRecommendationModel(nn.Module):
#     def __init__(self, user_dim, utterance_dim, exercise_dim, hidden_dim):
#         super(ExerciseRecommendationModel, self).__init__()
#         self.user_embedding = nn.Linear(user_dim, hidden_dim)
#         self.utterance_rnn = nn.GRU(utterance_dim, hidden_dim, batch_first=True)
#         self.exercise_embedding = nn.Linear(exercise_dim, hidden_dim)
#         self.attention = nn.MultiheadAttention(hidden_dim, num_heads=4)
#         self.fc = nn.Linear(hidden_dim * 3, 4)  # 4 output classes for 4 categories
        
#     def forward(self, user_features, utterance_features, exercise_features):
#         user_embedded = self.user_embedding(user_features)
#         utterance_output, _ = self.utterance_rnn(utterance_features)
#         exercise_embedded = self.exercise_embedding(exercise_features)
        
#         # Apply attention
#         attn_output, _ = self.attention(utterance_output, utterance_output, utterance_output)
        
#         # Concatenate user, utterance, and exercise features
#         combined = torch.cat([user_embedded, attn_output[:, -1, :], exercise_embedded], dim=1)
        
#         # Final classification
#         output = self.fc(combined)
#         return output


In [22]:

# def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs):
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model.to(device)
    
#     for epoch in range(num_epochs):
#         model.train()
#         train_loss = 0
#         for user_features, utterance_features, exercise_features in train_loader:
#             user_features = user_features.to(device)
#             utterance_features = utterance_features.to(device)
#             exercise_features = exercise_features.to(device)
            
#             optimizer.zero_grad()
#             outputs = model(user_features, utterance_features, exercise_features)
#             loss = criterion(outputs, exercise_features[:, 0].long())  # Assuming category is the first feature
#             loss.backward()
#             optimizer.step()
            
#             train_loss += loss.item()
        
#         # Val
#         model.eval()
#         val_loss = 0
#         with torch.no_grad():
#             for user_features, utterance_features, exercise_features in val_loader:
#                 user_features = user_features.to(device)
#                 utterance_features = utterance_features.to(device)
#                 exercise_features = exercise_features.to(device)
                
#                 outputs = model(user_features, utterance_features, exercise_features)
#                 val_loss += criterion(outputs, exercise_features[:, 0].long()).item()
        
#         print(f'Epoch {epoch+1}, Train Loss: {train_loss/len(train_loader):.4f}, Validation Loss: {val_loss/len(val_loader):.4f}')

In [23]:

# dataset = ExerciseRecommendationDataset(users_df, utterances_df, exercises_df)
# train_data, val_data = train_test_split(dataset, test_size=0.2, random_state=42)
# train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
# val_loader = DataLoader(val_data, batch_size=32)

In [24]:
# user_dim = 13  # Dimension of user features
# utterance_dim = 19  # Dimension of utterance features
# exercise_dim = 6  # Dimension of exercise features
# hidden_dim = 64  # Hidden dimension for the model

In [26]:
# model = ExerciseRecommendationModel(user_dim, utterance_dim, exercise_dim, hidden_dim)
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters())

# train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=25)

Epoch 1, Train Loss: 1.3200, Validation Loss: 1.1624
Epoch 2, Train Loss: 1.0614, Validation Loss: 0.9728
Epoch 3, Train Loss: 0.8959, Validation Loss: 0.8328
Epoch 4, Train Loss: 0.7744, Validation Loss: 0.7646
Epoch 5, Train Loss: 0.6851, Validation Loss: 0.6492
Epoch 6, Train Loss: 0.6020, Validation Loss: 0.5800
Epoch 7, Train Loss: 0.5236, Validation Loss: 0.5104
Epoch 8, Train Loss: 0.4664, Validation Loss: 0.4352
Epoch 9, Train Loss: 0.4085, Validation Loss: 0.3926
Epoch 10, Train Loss: 0.3514, Validation Loss: 0.3358
Epoch 11, Train Loss: 0.3112, Validation Loss: 0.2878
Epoch 12, Train Loss: 0.2665, Validation Loss: 0.2635
Epoch 13, Train Loss: 0.2320, Validation Loss: 0.2124
Epoch 14, Train Loss: 0.1997, Validation Loss: 0.1892
Epoch 15, Train Loss: 0.1707, Validation Loss: 0.1627
Epoch 16, Train Loss: 0.1448, Validation Loss: 0.1449
Epoch 17, Train Loss: 0.1270, Validation Loss: 0.1202
Epoch 18, Train Loss: 0.1085, Validation Loss: 0.1028
Epoch 19, Train Loss: 0.0954, Validat

In [27]:
# # Function to recommend exercise category
# def recommend_exercise_category(model, user, recent_utterances):
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model.to(device)
#     model.eval()
    
#     user_features = create_user_features(user).unsqueeze(0).to(device)
#     utterance_features = torch.stack([create_utterance_features(u) for u in recent_utterances]).unsqueeze(0).to(device)
#     dummy_exercise_features = torch.zeros(1, 6).to(device)  # Placeholder for exercise features
    
#     with torch.no_grad():
#         output = model(user_features, utterance_features, dummy_exercise_features)
    
#     category_probs = torch.softmax(output, dim=1)
#     recommended_category = torch.argmax(category_probs).item()
    
#     categories = ['grammar', 'vocabulary', 'pronunciation', 'fluency']
#     return categories[recommended_category]

In [29]:

# user_id = users_df['user_id'].iloc[0]
# user = users_df[users_df['user_id'] == user_id].iloc[0]
# recent_utterances = utterances_df[utterances_df['user_id'] == user_id].tail(5)
# recommended_category = recommend_exercise_category(model, user, recent_utterances.to_dict('records'))
# print(f"Recommended exercise category for user {user_id}: {recommended_category}")

Recommended exercise category for user e0fa472b-9738-4814-b366-5e0f6f668547: grammar


# FINAL ATTENTION MODEL

In [30]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np


def load_and_preprocess_data():
    users_df = pd.read_csv('users.csv')
    utterances_df = pd.read_csv('utterances.csv')
    errors_df = pd.read_csv('errors.csv')
    exercises_df = pd.read_csv('exercises.csv')
    user_progress_df = pd.read_csv('user_progress.csv')

    
    le_country = LabelEncoder()
    le_age_band = LabelEncoder()
    le_proficiency = LabelEncoder()
    le_cefr = LabelEncoder()
    le_current_focus = LabelEncoder()

    users_df['country_encoded'] = le_country.fit_transform(users_df['country'])
    users_df['age_band_encoded'] = le_age_band.fit_transform(users_df['age_band'])
    users_df['proficiency_level_encoded'] = le_proficiency.fit_transform(users_df['proficiency_level'])
    users_df['cefr_level_encoded'] = le_cefr.fit_transform(users_df['cefr_level'])
    users_df['current_focus_encoded'] = le_current_focus.fit_transform(users_df['current_focus'])

    users_df['interests'] = users_df['interests'].apply(eval)
    users_df['personalized_goals'] = users_df['personalized_goals'].apply(eval)

    scaler = StandardScaler()
    numerical_columns = ['overall_progress', 'total_practice_time', 'exercises_completed', 'daily_active_days', 
                         'weekly_active_days', 'total_sessions', 'avg_session_duration', 'adaptive_difficulty', 
                         'engagement_score']
    users_df[numerical_columns] = scaler.fit_transform(users_df[numerical_columns])

    
    utterance_scaler = StandardScaler()
    utterance_num_columns = ['grammar_overall_score', 'grammar_error_count', 'vocabulary_overall_score', 'unique_words_count',
                             'advanced_words_count', 'lexical_density', 'pronunciation_overall_score', 'phoneme_error_count',
                             'intonation_score', 'rhythm_score', 'accent_similarity', 'fluency_overall_score', 'words_per_minute',
                             'filler_word_count', 'pause_count', 'avg_pause_duration', 'longest_fluent_segment', 'coherence_score',
                             'topic_relevance_score']
    utterances_df[utterance_num_columns] = utterance_scaler.fit_transform(utterances_df[utterance_num_columns])

    
    le_category = LabelEncoder()
    le_subcategory = LabelEncoder()
    le_format = LabelEncoder()
    le_content_theme = LabelEncoder()

    exercises_df['category_encoded'] = le_category.fit_transform(exercises_df['category'])
    exercises_df['subcategory_encoded'] = le_subcategory.fit_transform(exercises_df['subcategory'])
    exercises_df['format_encoded'] = le_format.fit_transform(exercises_df['format'])
    exercises_df['content_theme_encoded'] = le_content_theme.fit_transform(exercises_df['content_theme'])

    exercise_scaler = StandardScaler()
    exercise_num_columns = ['difficulty', 'performance_score', 'time_taken']
    exercises_df[exercise_num_columns] = exercise_scaler.fit_transform(exercises_df[exercise_num_columns])

    return users_df, utterances_df, errors_df, exercises_df, user_progress_df, le_category, le_format, le_content_theme


class CombinedExerciseRecommendationModel(nn.Module):
    def __init__(self, user_dim, utterance_dim, exercise_dim, hidden_dim, num_categories, num_formats, num_themes):
        super(CombinedExerciseRecommendationModel, self).__init__()
        self.user_embedding = nn.Linear(user_dim, hidden_dim)
        self.utterance_rnn = nn.GRU(utterance_dim, hidden_dim, batch_first=True)
        self.exercise_embedding = nn.Linear(exercise_dim, hidden_dim)
        self.attention = nn.MultiheadAttention(hidden_dim, num_heads=4)
        self.fc_category = nn.Linear(hidden_dim * 3, num_categories)
        self.fc_format = nn.Linear(hidden_dim * 3, num_formats)
        self.fc_theme = nn.Linear(hidden_dim * 3, num_themes)
        
    def forward(self, user_features, utterance_features, exercise_features):
        user_embedded = self.user_embedding(user_features)
        utterance_output, _ = self.utterance_rnn(utterance_features)
        exercise_embedded = self.exercise_embedding(exercise_features)
        
        attn_output, _ = self.attention(utterance_output, utterance_output, utterance_output)
        
        combined = torch.cat([user_embedded, attn_output[:, -1, :], exercise_embedded], dim=1)
        
        category_output = self.fc_category(combined)
        format_output = self.fc_format(combined)
        theme_output = self.fc_theme(combined)
        
        return category_output, format_output, theme_output

# class for datset
class ExerciseRecommendationDataset(Dataset):
    def __init__(self, users, utterances, exercises, max_utterances=10):
        self.users = users
        self.utterances = utterances
        self.exercises = exercises
        self.max_utterances = max_utterances
        
    def __len__(self):
        return len(self.exercises)
    
    def __getitem__(self, idx):
        exercise = self.exercises.iloc[idx]
        user_id = exercise['user_id']
        user = self.users[self.users['user_id'] == user_id].iloc[0]
        
        user_features = torch.tensor([
            user['country_encoded'], user['age_band_encoded'], user['proficiency_level_encoded'],
            user['cefr_level_encoded'], user['current_focus_encoded'], user['overall_progress'],
            user['total_practice_time'], user['exercises_completed'], user['daily_active_days'],
            user['weekly_active_days'], user['total_sessions'], user['avg_session_duration'],
            user['adaptive_difficulty'], user['engagement_score']
        ], dtype=torch.float32)
        
        user_utterances = self.utterances[self.utterances['user_id'] == user_id].tail(self.max_utterances)
        utterance_features = torch.tensor(user_utterances[['grammar_overall_score', 'grammar_error_count',
                                                           'vocabulary_overall_score', 'unique_words_count',
                                                           'advanced_words_count', 'lexical_density',
                                                           'pronunciation_overall_score', 'phoneme_error_count',
                                                           'intonation_score', 'rhythm_score', 'accent_similarity',
                                                           'fluency_overall_score', 'words_per_minute',
                                                           'filler_word_count', 'pause_count', 'avg_pause_duration',
                                                           'longest_fluent_segment', 'coherence_score',
                                                           'topic_relevance_score']].values, dtype=torch.float32)
        
        if len(utterance_features) < self.max_utterances:
            padding = torch.zeros((self.max_utterances - len(utterance_features), utterance_features.size(1)))
            utterance_features = torch.cat([utterance_features, padding], dim=0)
        
        exercise_features = torch.tensor([
            exercise['category_encoded'], exercise['subcategory_encoded'], exercise['format_encoded'],
            exercise['content_theme_encoded'], exercise['difficulty'], exercise['performance_score'],
            exercise['time_taken']
        ], dtype=torch.float32)
        
        return user_features, utterance_features, exercise_features, exercise['category_encoded'], exercise['format_encoded'], exercise['content_theme_encoded']


def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device):
    model.to(device)
    
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for user_features, utterance_features, exercise_features, category, format_, theme in train_loader:
            user_features = user_features.to(device).float()
            utterance_features = utterance_features.to(device).float()
            exercise_features = exercise_features.to(device).float()
            category = category.to(device, dtype=torch.long)
            format_ = format_.to(device, dtype=torch.long)
            theme = theme.to(device, dtype=torch.long)
            
            optimizer.zero_grad()
            category_output, format_output, theme_output = model(user_features, utterance_features, exercise_features)
            
            loss = criterion(category_output, category) + criterion(format_output, format_) + criterion(theme_output, theme)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for user_features, utterance_features, exercise_features, category, format_, theme in val_loader:
                user_features = user_features.to(device).float()
                utterance_features = utterance_features.to(device).float()
                exercise_features = exercise_features.to(device).float()
                category = category.to(device ,dtype=torch.long)
                format_ = format_.to(device ,dtype=torch.long)
                theme = theme.to(device ,dtype=torch.long)
                
                category_output, format_output, theme_output = model(user_features, utterance_features, exercise_features)
                loss = criterion(category_output, category) + criterion(format_output, format_) + criterion(theme_output, theme)
                val_loss += loss.item()
        
        print(f'Epoch {epoch+1}, Train Loss: {train_loss/len(train_loader):.4f}, Validation Loss: {val_loss/len(val_loader):.4f}')

# recommendation function
def recommend_exercise(model, user, recent_utterances, users_df, utterances_df, le_category, le_format, le_content_theme, device):
    model.eval()
    
    user_features = torch.tensor([
        user['country_encoded'], user['age_band_encoded'], user['proficiency_level_encoded'],
        user['cefr_level_encoded'], user['current_focus_encoded'], user['overall_progress'],
        user['total_practice_time'], user['exercises_completed'], user['daily_active_days'],
        user['weekly_active_days'], user['total_sessions'], user['avg_session_duration'],
        user['adaptive_difficulty'], user['engagement_score']
    ], dtype=torch.float32).unsqueeze(0).to(device)

    utterance_features = torch.tensor(recent_utterances[['grammar_overall_score', 'grammar_error_count',
                                                         'vocabulary_overall_score', 'unique_words_count',
                                                         'advanced_words_count', 'lexical_density',
                                                         'pronunciation_overall_score', 'phoneme_error_count',
                                                         'intonation_score', 'rhythm_score', 'accent_similarity',
                                                         'fluency_overall_score', 'words_per_minute',
                                                         'filler_word_count', 'pause_count', 'avg_pause_duration',
                                                         'longest_fluent_segment', 'coherence_score',
                                                         'topic_relevance_score']].values, dtype=torch.float32).unsqueeze(0).to(device)

    dummy_exercise_features = torch.zeros(1, 7).to(device)  # Placeholder for exercise features

    with torch.no_grad():
        category_output, format_output, theme_output = model(user_features, utterance_features, dummy_exercise_features)

    category_probs = torch.softmax(category_output, dim=1)
    format_probs = torch.softmax(format_output, dim=1)
    theme_probs = torch.softmax(theme_output, dim=1)

    recommended_category = le_category.inverse_transform([torch.argmax(category_probs).item()])[0]
    recommended_format = le_format.inverse_transform([torch.argmax(format_probs).item()])[0]
    recommended_theme = le_content_theme.inverse_transform([torch.argmax(theme_probs).item()])[0]


    user_interests = user['interests']
    user_country = user['country']
    
    content_theme = np.random.choice(user_interests) if user_interests else recommended_theme
    
    # Simple logic for exercise format can be modified in diffrent ways- simple for brevity
    formats = ["fill-in-the-blank", "multiple-choice", "audio-recording", "sentence-formation"]
    format_weights = [0.3, 0.3, 0.2, 0.2]
    if recommended_category == 'pronunciation':
        format_weights = [0.1, 0.1, 0.7, 0.1]
    elif recommended_category == 'grammar':
        format_weights = [0.4, 0.4, 0.1, 0.1]
    
    exercise_format = np.random.choice(formats, p=format_weights)
    
    return {
        'category': recommended_category,
        'content_theme': content_theme,
        'format': exercise_format
    }


In [31]:

def main():
    users_df, utterances_df, errors_df, exercises_df, user_progress_df, le_category, le_format, le_content_theme = load_and_preprocess_data()
    
    dataset = ExerciseRecommendationDataset(users_df, utterances_df, exercises_df)
    train_data, val_data = train_test_split(dataset, test_size=0.2, random_state=42)
    train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=32)

    user_dim = 14  # Dimension of user features
    utterance_dim = 19  # Dimension of utterance features
    exercise_dim = 7  # Dimension of exercise features
    hidden_dim = 128  # Hidden dimension for the model
    num_categories = len(le_category.classes_)
    num_formats = len(le_format.classes_)
    num_themes = len(le_content_theme.classes_)

    model = CombinedExerciseRecommendationModel(user_dim, utterance_dim, exercise_dim, hidden_dim, num_categories, num_formats, num_themes)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters())
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    
    train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=100, device=device)

    
    user_id = users_df['user_id'].iloc[0]
    user = users_df[users_df['user_id'] == user_id].iloc[0]
    recent_utterances = utterances_df[utterances_df['user_id'] == user_id].tail(10)

    recommendation = recommend_exercise(model, user, recent_utterances, users_df, utterances_df, le_category, le_format, le_content_theme, device)
    print(f"Recommended exercise for user {user_id}:")
    print(recommendation)

   
    def update_user_progress(user_id, exercise_performance):
        user_index = users_df.index[users_df['user_id'] == user_id].tolist()[0]
        users_df.loc[user_index, 'overall_progress'] += exercise_performance['performance_score'] * 0.01
        users_df.loc[user_index, 'exercises_completed'] += 1
        users_df.loc[user_index, 'total_practice_time'] += exercise_performance['time_taken']
        users_df.loc[user_index, 'engagement_score'] += exercise_performance['performance_score'] * 0.005

        # recalculate the user's features
        user = users_df.loc[user_index]
        return user

    # simulating the recommendation system in action
    def simulate_user_session(user_id, num_exercises=20):
        print(f"Simulating session for user {user_id}")
        user = users_df[users_df['user_id'] == user_id].iloc[0]
        utterances_df = pd.read_csv('utterances.csv')  # Reload the utterances for simulation
        
        
        for i in range(num_exercises):
            print(f"\nExercise {i+1}")
            recent_utterances = utterances_df[utterances_df['user_id'] == user_id].tail(10)
            recommendation = recommend_exercise(model, user, recent_utterances, users_df, utterances_df, le_category, le_format, le_content_theme, device)
            print("Recommending exercise:")
            print(recommendation)
            
            # simulate exercise completion 
            performance_score = np.random.uniform(0.5, 1.0)
            time_taken = np.random.uniform(60, 300)  # seconds
            print(f"Simulated performance score: {performance_score:.2f}")
            print(f"Simulated time taken: {time_taken:.0f} seconds")
            
            # update user progress
            user = update_user_progress(user_id, {'performance_score': performance_score, 'time_taken': time_taken})
            
            # simulate a new utterance
            new_utterance = utterances_df[utterances_df['user_id'] == user_id].sample(1).iloc[0]
            new_utterance['timestamp'] = pd.Timestamp.now()
            utterances_df = pd.concat([utterances_df, new_utterance], ignore_index=True)

    
    random_user_id = np.random.choice(users_df['user_id'])
    
    simulate_user_session(random_user_id)

if __name__ == "__main__":
    main()

Epoch 1, Train Loss: 4.3742, Validation Loss: 3.7487
Epoch 2, Train Loss: 3.3947, Validation Loss: 3.1228
Epoch 3, Train Loss: 2.8695, Validation Loss: 2.6901
Epoch 4, Train Loss: 2.4918, Validation Loss: 2.4000
Epoch 5, Train Loss: 2.2527, Validation Loss: 2.2254
Epoch 6, Train Loss: 2.0014, Validation Loss: 1.9935
Epoch 7, Train Loss: 1.7950, Validation Loss: 1.7999
Epoch 8, Train Loss: 1.5702, Validation Loss: 1.5419
Epoch 9, Train Loss: 1.3904, Validation Loss: 1.4232
Epoch 10, Train Loss: 1.2341, Validation Loss: 1.1793
Epoch 11, Train Loss: 1.0937, Validation Loss: 1.0261
Epoch 12, Train Loss: 0.9706, Validation Loss: 1.0047
Epoch 13, Train Loss: 0.8884, Validation Loss: 0.9607
Epoch 14, Train Loss: 0.7687, Validation Loss: 0.8299
Epoch 15, Train Loss: 0.7039, Validation Loss: 0.7444
Epoch 16, Train Loss: 0.6296, Validation Loss: 0.6435
Epoch 17, Train Loss: 0.5751, Validation Loss: 0.6613
Epoch 18, Train Loss: 0.5355, Validation Loss: 0.6391
Epoch 19, Train Loss: 0.5131, Validat